In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import Select
import pandas as pd
from IPython.display import display
import warnings
import smtplib
import os
email=os.environ.get('EMAIL_ADDRESS')
password=os.environ.get('PASSWORD')
snu_email=os.environ.get('SNU_EMAIL')
snu_pass=os.environ.get('SNU_PASS')

In [10]:
course_letter1=list()
uwecourse_list=list()
print('''Note: Type 'done' when all courses are entered
Enter the UWE Course Codes of the subjects you are interested in
For Example type MAT if you want to get the details for math courses being offered''')

while True:
    uwe_course=input().upper().strip()
    if uwe_course == 'DONE':
        break
    elif len(uwe_course) != 3:
        print("Error: please enter a valid course code")
        continue
    else:
        uwecourse_list.append(uwe_course)

for x1 in uwecourse_list:
    if x1[:1] not in course_letter1:
        course_letter1.append(x1[:1])

Note: Type 'done' when all courses are entered
Enter the UWE Course Codes of the subjects you are interested in
For Example type MAT if you want to get the details for math courses being offered
eco
fac
done


In [4]:
warnings.filterwarnings('ignore')
browser = webdriver.Chrome('/Users/sanjaymalhotra/Desktop/Programming/Scraping/chromedriver')
time.sleep(3)
browser.get('https://prodweb.snu.in/psp/CSPROD/EMPLOYEE/HRMS/?cmd=login&languageCd=ENG&')
time.sleep(3)
browser.find_element_by_class_name('textnormal').click()
time.sleep(3)
elementID=browser.find_element_by_id('userid')
elementID.send_keys(snu_email)
elementID=browser.find_element_by_id('pwd')
elementID.send_keys(snu_pass)
browser.find_element_by_name('Submit').click()
time.sleep(3)
browser.find_element_by_class_name('SSSBUTTON_CONFIRMLINK').click()
time.sleep(3)
browser.get('https://prodweb.snu.in/psc/CSPROD_1/EMPLOYEE/HRMS/c/SA_LEARNER_SERVICES.SSS_BROWSE_CATLG_P.GBL?Page=SSS_BROWSE_CATLG&Action=U')

In [5]:
def AvailableSeats(lec):            
    for number in range(lec):
        temp_list1=list()
        browser.find_element_by_name('CLASS_SECTION$'+str(number)).click()
        time.sleep(2)
        seats_list=list()
        src6 = browser.page_source
        soup = BeautifulSoup(src6, 'lxml')
        seats=soup.find_all('span',class_='PSEDITBOX_DISPONLY')
        for seat_var in seats:
            seats_list.append(seat_var.text)

        time.sleep(2)
        available_seats=seats_list[-2]
        enrollment_total=seats_list[-4]
        total_seats=seats_list[-6]
        print(f'L-{number+1}: Total Seats: {total_seats}, Available Seats: {available_seats}')
        time.sleep(2)
        browser.find_element_by_id('CLASS_SRCH_WRK2_SSR_PB_CLOSE').click()
        time.sleep(2)

In [6]:
def Professor(prof_list,prof_list1,lec):
    src7 = browser.page_source
    soup = BeautifulSoup(src7, 'lxml') 
    prof=soup.find_all('span',class_='PSLONGEDITBOX')
    
    for p in prof:
        prof_list.append(p.text)
    print("Professors: ",end="")
    for pp in prof_list:
        if pp not in prof_list1 and len(pp)<25:
            prof_list1.append(pp)

    for ppp in prof_list1:
        try:
            print(prof_list1[lec-1],end=" ")
        except:
            print(prof_list1[0],end=" ")
        lec=lec-1
        if lec<=0:
            break
    print("\n")
    time.sleep(2)

In [11]:
list_for_finder=list()
list_for_finder1=list()
list3=list()

def finder(list_for_finder,list_for_finder1):
    warnings.filterwarnings('ignore')
    value_code=0
    for letter in list_for_finder1:
        time.sleep(2)
        browser.find_element_by_id('DERIVED_SSS_BCC_SSR_ALPHANUM_'+letter).click()
        time.sleep(2)
        src = browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        list1=list()
        list2=list()

        courses=soup.find_all('a',class_='SSSHYPERLINKBOLD')
        for c in courses:
            c=c.text.split()
            list1.append(c)
        for sub in list1:
            for subject in sub:
                if len(subject)==3 and subject not in list2 :
                    list2.append(subject)
        dict2=dict(list(enumerate(list2)))
        for k,v in dict2.items():
            if list_for_finder[value_code]==v:
                digit=k
        if 'SOC' in uwecourse_list or 'STM' in uwecourse_list or 'CHY' in uwecourse_list or 'COM' in uwecourse_list or 'CSD' in uwecourse_list:
            digit=digit+1
        
        value_code=value_code+1        
        time.sleep(2)        
        browser.find_element_by_id('DERIVED_SSS_BCC_GROUP_BOX_1$147$$'+str(digit)).click()
        course_list=list()

        time.sleep(2)
        src1 = browser.page_source
        soup = BeautifulSoup(src1, 'lxml')
        courses_offered=soup.find_all('a',class_='PSHYPERLINK')

        for uwe in courses_offered:
            course_list.append(uwe.text.strip())
        course_list = [x + y for x,y in zip(course_list[::2],course_list[1::2])]
        count=0
        total_ugcourses=0
        total_ugcourses1=0
        for ug in course_list:
            if int(ug[3:6])<500:
                count=count+1

        for e in range(count):
            terms_list=list()
            time.sleep(3)
            src5 = browser.page_source
            soup = BeautifulSoup(src5, 'lxml')
            browser.find_element_by_name('CRSE_TITLE$'+str(e)).click()
            time.sleep(2)

            try:
                browser.find_element_by_id('DERIVED_SAA_CRS_SSR_PB_GO').click()
            except:
                browser.find_element_by_name('DERIVED_SAA_CRS_RETURN_PB$163$').click()
                time.sleep(2)
                continue
            time.sleep(2)

            try:
                dropdown=browser.find_element_by_id('DERIVED_SAA_CRS_TERM_ALT')
                select=Select(dropdown)
                select.select_by_visible_text('Spring 2022')
                time.sleep(3)
                browser.find_element_by_name('DERIVED_SAA_CRS_SSR_PB_GO$3$').click()
                time.sleep(3)
            except:  
                src4 = browser.page_source
                soup = BeautifulSoup(src4, 'lxml')
                browser.find_element_by_name('DERIVED_SAA_CRS_RETURN_PB$163$').click()
                continue

            src2 = browser.page_source
            soup = BeautifulSoup(src2, 'lxml')
            termsoffered=soup.find_all('select',class_='PSDROPDOWNLIST')
            name_course=soup.find('span',class_='PALEVEL0SECONDARY')
            print(name_course.text[8:])
            list3.append(name_course.text[8:])
            
            timing=list()
            days_list=list()
            timing_list=list()
            src3 = browser.page_source
            soup = BeautifulSoup(src3, 'lxml')
            lec=soup.find_all('span',class_='PSEDITBOX_DISPONLY')
            for c in lec:
                timing.append((c.text.split()))
            for cc in timing:
                for ccc in cc:
                    if ':' in ccc:
                        timing_list.append(ccc)
            for cc1 in timing:
                for ccc1 in cc1:            
                    if ('Mo' in ccc1 or 'Tu' in ccc1 or 'We' in ccc1 or 'Th' in ccc1 or 'Fr' in ccc1 or 'Sa' in ccc1) and len(ccc1)<=4:
                        days_list.append(ccc1)
            out = [x + y for x,y in zip(timing_list[::2],timing_list[1::2])]
            time.sleep(3)

            lec_no=soup.find_all('a',class_='PSHYPERLINK')
            lec_list=list()
            for txt in lec_no:
                lec_list.append(txt.text)
            for aa in lec_list:
                if aa=='Return to Course List':
                    lec_list.remove(aa)
            days_count=0
            lec_count=0
            tut_count=0
            prac_count=0
            for aaa in lec_list:
                if 'LEC' in aaa:
                    lec_count=lec_count+1
                elif 'TUT' in aaa:
                    tut_count=tut_count+1
                elif 'PRA' in aaa:
                    prac_count=prac_count+1
            for aaaa in days_list:
                days_count=days_count+1
            lec=lec_count
        
            if lec_count+tut_count+prac_count != len(out):
                print('Error: Timings not proper')
                error=1
            else:
                error=0
            lec_count=days_count-tut_count-prac_count
            new_list=list()
            new_list1=list()
            for time_stamp in out[::-1]:
                if tut_count==0:
                    break
                else:
                    new_list.append('T-'+str(tut_count)+': '+time_stamp)
                    tut_count=tut_count-1
                    out.remove(time_stamp)
                
            for time_stamp in out[::-1]:
                if prac_count==0:
                    break
                else:
                    new_list.append('P-'+str(prac_count)+': '+time_stamp)
                    prac_count=prac_count-1 
 
            for time_stamp1 in out[:lec_count]:
                new_list1.append('L: '+time_stamp1)
                lec_count=lec_count-1
                if lec_count==0:
                    break
            new_list=new_list[::-1]
            new_list=new_list1+new_list

            time.sleep(3)
            df=pd.DataFrame([days_list,new_list])
            df.rename(index={1:name_course.text[8:]},inplace=True)
            df.rename(index={0:'Subject / Day'},inplace=True)
            new_header = df.iloc[0]
            df = df[1:]
            df.columns = new_header
            time.sleep(2)
            if error!=1:
                display(df)
            error=0
            time.sleep(2)

#Defined Function extracts details about the avaiable seats per lecture
            AvailableSeats(lec)

#Defined Function extracts details about the professors of each course
            prof_list=list()
            prof_list1=list()
            Professor(prof_list,prof_list1,lec)
            
            browser.find_element_by_name('DERIVED_SAA_CRS_RETURN_PB$163$').click()
            time.sleep(3)

In [12]:
finder(uwecourse_list,course_letter1)

ECO101 - Principles of Microeconomics
Error: Timings not proper
L-1: Total Seats: 90, Available Seats: 3
Professors: Dhritiman Gupta 

ECO102 - Principles of Macroeconomics
Error: Timings not proper
L-1: Total Seats: 80, Available Seats: 19
L-2: Total Seats: 80, Available Seats: 21
Professors: Suchismita Tarafdar 

ECO113 - Introduction to Probability


Subject / Day,TuTh,Mo,Th,Fr
ECO113 - Introduction to Probability,L: 15:3016:55,T-1: 19:0019:55,T-2: 17:0017:55,T-3: 16:0016:55


L-1: Total Seats: 80, Available Seats: 8
Professors: Shabana Mitra 

ECO203 - Introductory Econometrics


Subject / Day,MoWe,Fr,Mo,Tu
ECO203 - Introductory Econometrics,L: 14:0015:25,T-1: 19:0019:55,T-2: 19:0019:55,T-3: 17:0017:55


L-1: Total Seats: 150, Available Seats: 62
Professors: Ashokankur Datta 

ECO301 - Intermediate Microeconomics


...

L-1: Total Seats: 67, Available Seats: 5
Professors: Binay Chakrabarti 

FAC308 - Financial Markets & Institutions


Subject / Day,ThSa
FAC308 - Financial Markets & Institutions,L: 18:3019:55


L-1: Total Seats: 87, Available Seats: 0
Professors: Sunil Parameswaran 

FAC409 - Fintech


Subject / Day,TuFr
FAC409 - Fintech,L: 09:0010:25


L-1: Total Seats: 64, Available Seats: 8
Professors: Ankur Mehra 



In [14]:
with smtplib.SMTP('smtp.gmail.com',587) as smtp:
    smtp.ehlo()
    smtp.starttls()
    smtp.ehlo()  
    smtp.login(email,password)
    subject='SNU: UWE Courses'
    body=f'The list of UWEs Offered from {uwecourse_list} in Spring 2022 is:\n{list3}'
    msg=f'Subject: {subject}\n\n{body}'
    smtp.sendmail(email,email,msg)
print("List of all courses successfully mailed to the user")

List of all courses successfully mailed to the user
